In [3]:
%load_ext autoreload
%autoreload 2

from Pytorch_VAE_LSTM import VAE
from dataset.modules.music_dataset import TorchMusicDataset

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import matplotlib.pyplot as plt
import numpy as np

dataset = TorchMusicDataset("./dataset/cleaned_data/cleaned_data")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
songs = []
for i in range(700):
    obj = dataset.__getitem__(i)
    song = obj['time_series'].numpy()[:-7]
    reduce_mean_song = np.mean(song.reshape(-1, 40), axis=1) 
    songs.append(reduce_mean_song.reshape(reduce_mean_song.shape[0], 1))

In [5]:
songs[0].shape

(33049, 1)

In [6]:
batch_size = 16
n_in = songs[0].shape[0]
songs = np.array(songs)
songs = torch.tensor(songs, dtype=torch.float32)

dataset = TensorDataset(songs, songs) 
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

vae = VAE(timesteps=n_in, features=1, latent_dim=4, device=device).to(device)
vae.optimizer = optim.Adam(vae.parameters(), lr=1e-3)

# Training loop
epochs = 300
for epoch in range(epochs):
    for batch_x, _ in dataloader:
        batch_x = batch_x.to(device)
        loss = vae.training_step(batch_x)
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss}')
    
num_samples = 1
generated_sequences = vae.generate_sequences(num_samples)

Epoch 0, Loss: 0.07434827089309692
Epoch 10, Loss: 0.03756450489163399
Epoch 20, Loss: 0.023068726062774658
Epoch 30, Loss: 0.04853392392396927
Epoch 40, Loss: 0.03283509239554405
Epoch 50, Loss: 0.021407639607787132
Epoch 60, Loss: 0.024609077721834183
Epoch 70, Loss: 0.0228077732026577
Epoch 80, Loss: 0.02469830960035324
Epoch 90, Loss: 0.031313106417655945
Epoch 100, Loss: 0.040324315428733826
Epoch 110, Loss: 0.04006321355700493
Epoch 120, Loss: 0.04513774812221527
Epoch 130, Loss: 0.03620599955320358
Epoch 140, Loss: 0.030614370480179787
Epoch 150, Loss: 0.02321167103946209
Epoch 160, Loss: 0.04040929302573204
Epoch 170, Loss: 0.029920071363449097
Epoch 180, Loss: 0.027222048491239548
Epoch 190, Loss: 0.0470939576625824
Epoch 200, Loss: 0.020496850833296776
Epoch 210, Loss: 0.03050239384174347
Epoch 220, Loss: 0.027865465730428696
Epoch 230, Loss: 0.03592591732740402
Epoch 240, Loss: 0.029831642284989357
Epoch 250, Loss: 0.03239224851131439
Epoch 260, Loss: 0.02272106148302555
Epo

In [10]:
reconstructed_songs = []
for sequence in generated_sequences:
    reconstructed_song = np.repeat(reduce_mean_song, 40, axis=0)
    reconstructed_songs.append(reconstructed_song)

In [12]:
reconstructed_songs[0]

array([-1.0124829e-06, -1.0124829e-06, -1.0124829e-06, ...,
       -3.6045098e-01, -3.6045098e-01, -3.6045098e-01], dtype=float32)

In [24]:
import numpy as np
from scipy.io.wavfile import write

# Example usage
audio_array = reconstructed_songs[0]  # Replace with your actual audio array
sample_rate = 44100  # Replace with your actual sample rate

# Normalize the audio array to be within the range [-1, 1]
audio_array = audio_array / np.max(np.abs(audio_array))

# Convert the normalized audio array to 16-bit PCM format
audio_array = (audio_array * 32767).astype(np.int16)

# Save the audio array as a .wav file
output_filename = 'output_audio.wav'
write(output_filename, sample_rate, audio_array)

print(f"Audio saved as {output_filename}")

Audio saved as output_audio.wav
